<a href="https://colab.research.google.com/github/wtaisner/tensorflow-great-barrier-reef/blob/main/FastRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import wandb
except:
    !pip install wandb
    import wandb
!wandb login

     |████████████████████████████████| 1.7 MB 8.0 MB/s 
     |████████████████████████████████| 180 kB 65.1 MB/s 
     |████████████████████████████████| 142 kB 69.3 MB/s 
     |████████████████████████████████| 97 kB 9.4 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=314de812ddbec36da4724940687b8c4866a32f80d9f1da04adbb3c3b6b2842a3
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=3d11e8a56e233f0e59689301e58ae7730046250255dc914186f9b8b51794d81c
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or p

In [2]:
try:
    import torchmetrics
except:
    !pip install torchmetrics
    import torchmetrics

     |████████████████████████████████| 396 kB 8.0 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision
import ast
import torch
from tqdm import tqdm

from torchmetrics.detection.map import MeanAveragePrecision


# this should probably be changed to something smart, right?
KAGGLE_PATH_ANNOTATIONS = '/kaggle/input/tensorflow-great-barrier-reef/train.csv'
KAGGLE_PATH_IMG_DIR = '/kaggle/input/tensorflow-great-barrier-reef/train_images/'
LOCAL_PATH_ANNOTATIONS = 'data/train.csv'
LOCAL_PATH_IMG_DIR = 'data/train_images/'
COLAB_PATH_ANNOTATIONS = '/content/drive/MyDrive/data/train.csv'
COLAB_PATH_IMG_DIR = '/content/drive/MyDrive/data/train_images/'

wandb.config = {
  "learning_rate": 0.001,
  "epochs": 2,
  "batch_size": 2,
  "momentum": 0.9,
  "weight_decay": 0.0005, 
  "confidence_threshold": 0.5 # save a bounding box if model returned confidence above this threshold
}

In [5]:
class StarfishDataset(Dataset):
    def __init__(self,
                 annotations_file=COLAB_PATH_ANNOTATIONS,
                 img_dir=COLAB_PATH_IMG_DIR
                 ):
        self.img_labels = pd.read_csv(annotations_file)
        self.annotated = self.img_labels[self.img_labels['annotations'] != '[]']  # get only annotated frames
        self.img_dir = img_dir

    def __len__(self):
        return len(self.annotated)

    def __getitem__(self, idx):
        image = read_image(os.path.join(self.img_dir, 'video_{}'.format(self.annotated.iloc[idx][0]),
                                        '{}.jpg'.format(self.annotated.iloc[idx][2])))
        min_image = image.min()
        max_image = image.max()
        # normalize image to 0-1 - required by torchvision
        image -= min_image
        image = torch.FloatTensor(image/max_image)
        labels = self.annotated.iloc[idx][-1]
        labels = ast.literal_eval(labels)
        coords = []
        for parsed_label in labels:
            x1, y1 = parsed_label['x'], parsed_label['y']
            x2, y2 = x1+parsed_label['width'], y1+parsed_label['height']
            coords.append([x1, y1, x2, y2])

        boxes = torch.FloatTensor(coords)
        labels = torch.LongTensor([1 for _ in range(len(coords))]) # label has to be integer, since we have only one label I coded it as 1 for simplicity
        return image, boxes, labels

# dataset = StarfishDataset()
# dataset.__getitem__(0)


In [6]:
def collate_fn(batch):
    targets = []
    images = []
    for imgs, boxes, labels in batch:
        images.append(imgs)
        d = {}
        d['boxes'] = boxes
        d['labels'] = labels
        targets.append(d)
    return images, targets

def slice_output(output: dict, confidence_threshold: float = wandb.config['confidence_threshold']) -> dict:
    """
    this method is responsible for validating models output w.r.t confidence_threshold defined above.
    It accepts an output dictionary from model, namely {'boxes':[], 'labels':[], 'scores':[]}
    It returns a dictionary sliced to items with score above confidence_threshold
    """

    num_valid_elements = np.sum(np.array(output['scores']) >= confidence_threshold)
    # temporary option to make sure, that it returns at least one element, although it should probably be fixed,
    # should there be any frames where there is no starfish
    if num_valid_elements == 0:
        num_valid_elements = 1
    res = {}
    for key, value in output.items():
        res[key] = value[:num_valid_elements]
    return res

In [7]:
torch.manual_seed(23)

# IF YOU WANT TO RUN PROPER MODEL LEARNING, MAKE SURE TO CHANGE DATASET SIZES

dataset = StarfishDataset()
train_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# extract only small part of the data for faster learning / testing process
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, test_size])


print('Train dataset: {} instances, test dataset: {}'.format(len(train_dataset), len(test_dataset)))


train_dataloader = DataLoader(
    train_dataset, batch_size=wandb.config['batch_size'], shuffle=False, num_workers=1, collate_fn = collate_fn)
test_dataloader = DataLoader(
    test_dataset, batch_size=wandb.config['batch_size'], shuffle=False, num_workers=1,  collate_fn = collate_fn)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
cpu = torch.device('cpu')
print('Used device: {}'.format(device))

num_classes = 2  # starfish and not starfish I guess

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=wandb.config['learning_rate'], momentum=wandb.config['momentum'], weight_decay=wandb.config['weight_decay'])
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

Train dataset: 392 instances, test dataset: 99
Used device: cuda


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [8]:
# https://pytorch.org/vision/stable/models.html#runtime-characteristics see Faster R-CNN for the details of this model, what it requires, returns, etc

# https://github.com/pytorch/vision/blob/main/references/detection/engine.py probably see training and eval loops here

wandb.init(project="great-barrier-reef", entity="ap-wt", config = wandb.config)

wandb.watch(model, log="all")

for e in tqdm(range(wandb.config['epochs'])):
    print('\n')
        
    model.train()

    for idx, (images, targets) in enumerate(train_dataloader):

        images = list(image.to(device) for image in images)

        for d in targets:
            d['boxes'] = d['boxes'].to(device)
            d['labels'] = d['labels'].to(device)

        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    lr_scheduler.step()
            
    model.eval()
    with torch.no_grad():
        for idx, (images, targets) in enumerate(test_dataloader):

            images = list(image.to(device) for image in images)
            predictions = model(images)
            outputs = [{k: v.to(cpu) for k, v in t.items()} for t in predictions]

            # TODO: add some comparison with 'targets' perhaps
            # TODO: any loss functions that is more reliable than this ? idk
            outputs = [slice_output(out) for out in outputs]
            metric = MeanAveragePrecision()
            metric.update(outputs, targets)
            metrics = metric.compute()
            if idx % 100 == 0:
                wandb.log({'MAP':metrics['map'], 'MAR_1':metrics['mar_1']})
        


    optimizer.step()
wandb.finish()

wandb: Currently logged in as: ap-wt (use `wandb login --relogin` to force relogin)


  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 50%|█████     | 1/2 [03:20<03:20, 200.02s/it]

100%|██████████| 2/2 [05:41<00:00, 170.56s/it]


MAP,▁█
MAR_1,▁█
MAP,0.30297
MAR_1,0.3


In [9]:
# torch.save(model.state_dict(), 'models/FastRCNN.pt')

FileNotFoundError: ignored